In [ ]:
import pandas as pd
import re
import numpy as np

from datetime import datetime

from pathlib import Path
import json

from IPython.display import display

In [5]:
df = pd.read_csv("../Resources/datasets/all_events_merged.csv")

In [6]:
df.head()

,title,date,time,venue,place,price,url
0,NaN,NaN,NaN,NaN,NaN,NaN,https://in.bookmyshow.com/events/2-dogle/ET004...
1,3D magic Canvas painting,2025-10-01,15:30,NaN,NaN,999.0,https://in.bookmyshow.com/events/3d-magic-canv...
2,A Multi Sensory A/V Show,2025-10-18,22:00,NaN,NaN,499.0,https://in.bookmyshow.com/events/a-multi-senso...
3,Aaj Rang Hai - Nizami Brothers Live,2025-10-12,15:30,NaN,NaN,500.0,https://in.bookmyshow.com/events/aaj-rang-hai-...
4,Ab Hai Aapki Bari by Inder Sahani,2025-10-02,16:00,NaN,NaN,699.0,https://in.bookmyshow.com/events/aap-manoge-na...


In [7]:
df.tail()

,title,date,time,venue,place,price,url
576,The Cry Club,2025-10-04,17:35,NaN,NaN,499.0,https://in.bookmyshow.com/events/listener-s-ci...
577,Little Green Thumbs,2025-10-11,10:00,NaN,NaN,1800.0,https://in.bookmyshow.com/events/little-green-...
578,Lollapalooza India 2026,2026-01-24,14:00,NaN,NaN,49999.0,https://in.bookmyshow.com/events/lollapalooza-...
579,Macrame Workshop,2025-10-05,18:00,NaN,NaN,1599.0,https://in.bookmyshow.com/events/macrame-works...
580,Kid's Texture Workshop,2025-10-05,18:00,NaN,NaN,1699.0,https://in.bookmyshow.com/events/macrame-works...


In [8]:
print(df.shape)

(581, 7)


In [ ]:
print(df.columns.tolist())

['title', 'date', 'time', 'venue', 'place', 'price', 'url']


In [11]:
def normalize_col(col: str) -> str:
    c = str(col).strip()
    c = c.replace(" ", "_").replace(".", "_")
    c = re.sub(r"[^0-9a-zA-Z_]+", "_", c)
    c = re.sub(r"__+", "_", c)
    c = c.strip("_")
    return c.lower()

In [12]:
# normalize column names
df.columns = [normalize_col(c) for c in df.columns]

In [13]:
# standardize expected columns if present
rename_map = {
    "event_title": "title",
    "name": "title",
    "event_name": "title",
    "price_range": "price",
}

df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

In [15]:
# trim whitespace in strings
for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip().replace({"nan": np.nan, "none": np.nan, "": np.nan})

df.head()

,title,date,time,venue,place,price,url
0,NaN,NaN,NaN,NaN,NaN,NaN,https://in.bookmyshow.com/events/2-dogle/ET004...
1,3D magic Canvas painting,2025-10-01,15:30,NaN,NaN,999.0,https://in.bookmyshow.com/events/3d-magic-canv...
2,A Multi Sensory A/V Show,2025-10-18,22:00,NaN,NaN,499.0,https://in.bookmyshow.com/events/a-multi-senso...
3,Aaj Rang Hai - Nizami Brothers Live,2025-10-12,15:30,NaN,NaN,500.0,https://in.bookmyshow.com/events/aaj-rang-hai-...
4,Ab Hai Aapki Bari by Inder Sahani,2025-10-02,16:00,NaN,NaN,699.0,https://in.bookmyshow.com/events/aap-manoge-na...


In [17]:
# Ensure date/time columns exist (create empty if missing)
if "date" not in df.columns:
    df["date"] = pd.NaT
if "time" not in df.columns:
    df["time"] = np.nan

In [18]:
# Normalize time strings like "22:00", "10:30 PM", "7pm"
def normalize_time_str(x: str):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    s = s.replace(".", "")
    s = s.replace("hrs","").replace("hr","").strip()
    # Common variants
    if re.fullmatch(r"\d{1,2}$", s):
        # "7" -> "07:00"
        return f"{int(s):02d}:00"
    if re.fullmatch(r"\d{1,2}:\d{2}$", s):
        return s
    # am/pm handling
    try:
        dt = pd.to_datetime(s, format="%I%p", errors="coerce")
        if pd.notna(dt): return dt.strftime("%H:%M")
    except: pass
    try:
        dt = pd.to_datetime(s, format="%I:%M%p", errors="coerce")
        if pd.notna(dt): return dt.strftime("%H:%M")
    except: pass
    # Let pandas try best-effort
    dt = pd.to_datetime(s, errors="coerce")
    if pd.notna(dt):
        return dt.strftime("%H:%M")
    return np.nan

df["time_norm"] = df["time"].map(normalize_time_str)

In [19]:
# Build combined datetime (assumes date is ISO-like or parseable)
date_parsed = pd.to_datetime(df["date"], errors="coerce", utc=False, infer_datetime_format=True)
# If time missing, set 00:00
time_filled = df["time_norm"].fillna("00:00")

# Combine safely
def combine_dt(d, t):
    if pd.isna(d): return pd.NaT
    try:
        return pd.to_datetime(f"{d.date()} {t}", errors="coerce")
    except Exception:
        return pd.NaT

df["start_dt_local"] = [combine_dt(d, t) for d, t in zip(date_parsed, time_filled)]

C:\Users\LEGION\AppData\Local\Temp\ipykernel_26072\46724713.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  date_parsed = pd.to_datetime(df["date"], errors="coerce", utc=False, infer_datetime_format=True)


In [20]:
df["start_date"] = df["start_dt_local"].dt.date

df[["date","time","time_norm","start_dt_local","start_date"]].head(8)

,date,time,time_norm,start_dt_local,start_date
0,NaN,NaN,NaN,NaT,NaT
1,2025-10-01,15:30,15:30,2025-10-01 15:30:00,2025-10-01
2,2025-10-18,22:00,22:00,2025-10-18 22:00:00,2025-10-18
3,2025-10-12,15:30,15:30,2025-10-12 15:30:00,2025-10-12
4,2025-10-02,16:00,16:00,2025-10-02 16:00:00,2025-10-02
5,2025-10-31,19:00,19:00,2025-10-31 19:00:00,2025-10-31
6,2025-10-11,18:00,18:00,2025-10-11 18:00:00,2025-10-11
7,2025-12-20,19:00,19:00,2025-12-20 19:00:00,2025-12-20


In [21]:
def parse_price_range(s):
    if pd.isna(s): return (np.nan, np.nan)
    txt = str(s)
    # Replace comma thousand separators; keep digits, dot, dash
    cleaned = re.sub(r"[^\d\.\-–—]", " ", txt)
    # Normalize dashes
    cleaned = cleaned.replace("–", "-").replace("—", "-")
    nums = re.findall(r"\d+(?:\.\d+)?", cleaned)
    if not nums:
        return (np.nan, np.nan)
    vals = [float(n) for n in nums]
    if "-" in cleaned and len(vals) >= 2:
        return (min(vals[0], vals[1]), max(vals[0], vals[1]))
    # Single value
    return (vals[0], vals[0])

if "price" in df.columns:
    pm, px = zip(*df["price"].map(parse_price_range))
    df["price_min"] = pm
    df["price_max"] = px
else:
    df["price_min"] = np.nan
    df["price_max"] = np.nan

df[["price","price_min","price_max"]].head(8)


,price,price_min,price_max
0,NaN,NaN,NaN
1,999.0,999.0,999.0
2,499.0,499.0,499.0
3,500.0,500.0,500.0
4,699.0,699.0,699.0
5,599.0,599.0,599.0
6,399.0,399.0,399.0
7,3000.0,3000.0,3000.0


In [ ]:
# Many feeds use "Place" as "City, State, Country" or "Neighborhood, City"
for col in ["venue","place"]:
    if col not in df.columns:
        df[col] = np.nan

def split_place(p):
    if pd.isna(p): return (np.nan, np.nan, np.nan)
    parts = [x.strip() for x in str(p).split(",") if x.strip()]
    if not parts: return (np.nan, np.nan, np.nan)
    # Heuristic: last -> country (if looks like 2-3 letter code or known country word length)
    if len(parts) == 1:
        return (parts[0], np.nan, np.nan)  # city only
    if len(parts) == 2:
        return (parts[0], parts[1], np.nan)  # city, state
    # 3 or more: assume ... , city, state, country
    return (parts[-3], parts[-2], parts[-1])

df[["city","state","country"]] = df.apply(
    lambda r: pd.Series(split_place(r.get("place"))),
    axis=1
)

df[["venue","place","city","state","country"]].head(10)

,venue,place,city,state,country
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [23]:
# Canonical subset
canonical_cols = [
    c for c in [
        "title","start_dt_local","start_date",
        "venue","city","state","country",
        "price_min","price_max","place","price"
    ] if c in df.columns
]
canon = df[canonical_cols].copy()

# Dedup key: title + date + venue + city
def make_key(row):
    parts = []
    for c in ["title","start_date","venue","city"]:
        v = row.get(c)
        if pd.notna(v):
            parts.append(str(v).lower())
    return "||".join(parts) if parts else np.nan

canon["_key"] = canon.apply(make_key, axis=1)
before = len(canon)
canon = canon.drop_duplicates(subset=["_key"]).drop(columns=["_key"])
after = len(canon)
print(f"Deduplicated {before - after} rows")

canon.head(10)


Deduplicated 143 rows


,title,start_dt_local,start_date,venue,city,state,country,price_min,price_max,place,price
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3D magic Canvas painting,2025-10-01 15:30:00,2025-10-01,NaN,NaN,NaN,NaN,999.0,999.0,NaN,999.0
2,A Multi Sensory A/V Show,2025-10-18 22:00:00,2025-10-18,NaN,NaN,NaN,NaN,499.0,499.0,NaN,499.0
3,Aaj Rang Hai - Nizami Brothers Live,2025-10-12 15:30:00,2025-10-12,NaN,NaN,NaN,NaN,500.0,500.0,NaN,500.0
4,Ab Hai Aapki Bari by Inder Sahani,2025-10-02 16:00:00,2025-10-02,NaN,NaN,NaN,NaN,699.0,699.0,NaN,699.0
5,Telling Lies -A Standup Solo by Aashish Solanki,2025-10-31 19:00:00,2025-10-31,NaN,NaN,NaN,NaN,599.0,599.0,NaN,599.0
6,Ab Tumse Kya Chupana Ft- Mohit Dudeja,2025-10-11 18:00:00,2025-10-11,NaN,NaN,NaN,NaN,399.0,399.0,NaN,399.0
7,Abhijeet Bhattacharya: Live in Concert Gurugram,2025-12-20 19:00:00,2025-12-20,NaN,NaN,NaN,NaN,3000.0,3000.0,NaN,3000.0
8,Acrylic Painting By Social Sozo,2025-10-01 16:00:00,2025-10-01,NaN,NaN,NaN,NaN,799.0,799.0,NaN,799.0
9,Acrylic Affair,2025-10-04 15:00:00,2025-10-04,NaN,NaN,NaN,NaN,699.0,699.0,NaN,699.0


In [25]:
OUT_DIR = Path("./clean_outputs")  # change if you like
OUT_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_CSV   = OUT_DIR / "events_clean_basic.csv"
REPORT_JSON = OUT_DIR / "events_clean_basic_report.json"

canon.to_csv(CLEAN_CSV, index=False)

report = {
    "input_shape": [int(df.shape[0]), int(df.shape[1])],
    "output_shape": [int(canon.shape[0]), int(canon.shape[1])],
    "dedup_removed": int(before - after),
    "columns_present": canon.columns.tolist(),
    "notes": "Basic clean: headers, start datetime, price min/max, city/state/country (heuristic), dedup."
}
with open(REPORT_JSON, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)

print("Saved:")
print(f"- {CLEAN_CSV}")
print(f"- {REPORT_JSON}")

Saved:
- clean_outputs\events_clean_basic.csv
- clean_outputs\events_clean_basic_report.json


In [ ]:
# Dates
if "start_date" in canon.columns:
    canon["start_date"] = pd.to_datetime(canon["start_date"], errors="coerce").dt.normalize()

# Prices
if "price_min" in canon.columns:
    canon["price_min"] = pd.to_numeric(canon["price_min"], errors="coerce")
if "price_max" in canon.columns:
    canon["price_max"] = pd.to_numeric(canon["price_max"], errors="coerce")

# Preview
display(canon.head(25))

# Top cities
print("\nTop cities:")
if "city" in canon.columns:
    print(canon["city"].value_counts(dropna=False).head(10))

# Date range
print("\nDate range:")
if "start_date" in canon.columns:
    sd = canon["start_date"]
    if sd.notna().any():
        print(sd.min().date(), "->", sd.max().date())
    else:
        print("No valid dates found")
else:
    print("start_date column not present")

# Price summary
print("\nPrice summary:")
if "price_min" in canon.columns:
    pm = canon["price_min"]
    if pm.notna().any():
        print("min:", float(pm.min()), " | max:", float(pm.max()))
    else:
        print("No numeric values in price_min")
else:
    print("price_min column not present")


,title,start_dt_local,start_date,venue,city,state,country,price_min,price_max,place,price
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3D magic Canvas painting,2025-10-01 15:30:00,2025-10-01,NaN,NaN,NaN,NaN,999.0,999.0,NaN,999.0
2,A Multi Sensory A/V Show,2025-10-18 22:00:00,2025-10-18,NaN,NaN,NaN,NaN,499.0,499.0,NaN,499.0
3,Aaj Rang Hai - Nizami Brothers Live,2025-10-12 15:30:00,2025-10-12,NaN,NaN,NaN,NaN,500.0,500.0,NaN,500.0
4,Ab Hai Aapki Bari by Inder Sahani,2025-10-02 16:00:00,2025-10-02,NaN,NaN,NaN,NaN,699.0,699.0,NaN,699.0
5,Telling Lies -A Standup Solo by Aashish Solanki,2025-10-31 19:00:00,2025-10-31,NaN,NaN,NaN,NaN,599.0,599.0,NaN,599.0
6,Ab Tumse Kya Chupana Ft- Mohit Dudeja,2025-10-11 18:00:00,2025-10-11,NaN,NaN,NaN,NaN,399.0,399.0,NaN,399.0
7,Abhijeet Bhattacharya: Live in Concert Gurugram,2025-12-20 19:00:00,2025-12-20,NaN,NaN,NaN,NaN,3000.0,3000.0,NaN,3000.0
8,Acrylic Painting By Social Sozo,2025-10-01 16:00:00,2025-10-01,NaN,NaN,NaN,NaN,799.0,799.0,NaN,799.0
9,Acrylic Affair,2025-10-04 15:00:00,2025-10-04,NaN,NaN,NaN,NaN,699.0,699.0,NaN,699.0



Top cities:
city
NaN    438
Name: count, dtype: int64

Date range:
2025-09-30 -> 2026-03-15

Price summary:
min: 0.0  | max: 590000.0


In [ ]:
# Ensure clean dates right after creation
df["start_dt_local"] = pd.to_datetime(df["start_dt_local"], errors="coerce")
df["start_date"] = pd.to_datetime(df["start_dt_local"], errors="coerce").dt.normalize()